## Simulation script for exploration of missing data strategies

In this script, we perform simulations to explore the effect of several missing data techniques on the outcome of a regression analysis. We work with two real, example datasets (imported with file: import_real_data.ipynb) and six custom datasets (created in the file: generate_data.ipynb). We developed an amputation function and generated multiple forms of missing data. We deal with the missing data by the following missing data methods: drop, mean imputation, median imputation, regression imputation, stochastic regression imputation and random imputation. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

pd.set_option("max_rows",250)
pd.set_option("max_columns",999)

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as ev

## Amputation function

In [2]:
# Function to delete entries of dataframe
def delete_data(df_,mechanism,output_variable,missing_row_proportion=0.05):
    
    df = df_.copy()
    index_ = df.index.values
    all_columns = df.drop(output_variable,1).columns.tolist()
    total_rows = int(len(df)*missing_row_proportion)
    cells_per_column = int(total_rows / 2) #50% of cells in selected rows are made missing
    
    index_selected = np.random.choice(index_,total_rows,replace=False)       
 
    if mechanism == 'MCAR':        
        for col in all_columns:
            drop_index = np.random.choice(index_selected,cells_per_column,replace=False)
            df[col].loc[drop_index] = None
            
    elif mechanism == 'MAR':
        #records with a high y value receive a large weight
        weights = df.loc[index_selected][output_variable].values+\
        abs(df.loc[index_selected][output_variable].values.min())+0.05 
        #we add 0.05 to give each cell a small chance of being missing
        for col in all_columns:
            drop_index = np.random.choice(index_selected,size=cells_per_column,
                                          p=weights/weights.sum(axis=0),replace=False)
            df[col].loc[drop_index] = None
            
    elif mechanism == 'MNAR':        
        for col in all_columns:
            #records with a high value on that specific feature receive a large weight
            weights = df.loc[index_selected][col].values+abs(df.loc[index_selected][col].values.min())+0.05
            #we add 0.05 to give each cell a small chance of being missing
            drop_index = np.random.choice(index_selected,size=cells_per_column,
                                          p=weights/weights.sum(axis=0),replace=False)
            df[col].loc[drop_index] = None

    return df

# Missing data strategies

In [3]:
def apply_train_test(data,test_size=0.4):
    train, test = train_test_split(data.index.values,test_size=test_size)
    return train, test

In [14]:
def apply_drop(train,test,model):
    train_use = train.dropna().copy()
    test_use = test.dropna().copy()
    mod = model.fit(train_use.drop(output_variable,1),train_use[output_variable])
    pred = mod.predict(test_use.drop(output_variable,1))
    return {'mse': mse(test_use[output_variable],pred), 'ev': ev(test_use[output_variable],pred),
           'mae': mae(test_use[output_variable],pred)}

In [16]:
import numpy as np
import numpy.ma as ma
from scipy import sparse
from scipy import stats

from sklearn.preprocessing.imputation import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.utils.sparsefuncs import _get_median
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import FLOAT_DTYPES

import warnings

import numpy as np
import numpy.ma as ma
from scipy import sparse
from scipy import stats

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.utils.sparsefuncs import _get_median
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import FLOAT_DTYPES

from sklearn.externals import six

zip = six.moves.zip
map = six.moves.map

__all__ = [
    'Imputer',
]

def _get_mask(X, value_to_mask):
    """Compute the boolean mask X == missing_values."""
    if value_to_mask == "NaN" or np.isnan(value_to_mask):
        return np.isnan(X)
    else:
        return X == value_to_mask

class custom_imputer(Imputer):
        
    def fit(self, X, y=None):
        """Fit the imputer on X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            Input data, where ``n_samples`` is the number of samples and
            ``n_features`` is the number of features.
        Returns
        -------
        self : Imputer
            Returns self.
        """

        # Check parameters
        allowed_strategies = ["mean", "median", "most_frequent", "random", "regression", "stochastic"] 
        ## ADDED RANDOM METHOD 
        if self.strategy not in allowed_strategies:
            raise ValueError("Can only use these strategies: {0} "
                             " got strategy={1}".format(allowed_strategies,
                                                        self.strategy))

        if self.axis not in [0, 1]:
            raise ValueError("Can only impute missing values on axis 0 and 1, "
                             " got axis={0}".format(self.axis))

        # Since two different arrays can be provided in fit(X) and
        # transform(X), the imputation data will be computed in transform()
        # when the imputation is done per sample (i.e., when axis=1).
        if self.axis == 0:
            X = check_array(X, accept_sparse='csc', dtype=np.float64,
                            force_all_finite=False)      

            if sparse.issparse(X):
                self.statistics_ = self._sparse_fit(X,
                                                    self.strategy,
                                                    self.missing_values,
                                                    self.axis)
            else:
                self.statistics_ = self._dense_fit(X,
                                                   self.strategy,
                                                   self.missing_values,
                                                   self.axis)
        return self

    def _dense_fit(self, X, strategy, missing_values, axis):
        """Fit the transformer on dense data."""
        X = check_array(X, force_all_finite=False)
        mask = _get_mask(X, missing_values)
        masked_X = ma.masked_array(X, mask=mask)

        # Mean
        if strategy == "mean":
            mean_masked = np.ma.mean(masked_X, axis=axis)
            # Avoid the warning "Warning: converting a masked element to nan."
            mean = np.ma.getdata(mean_masked)
            mean[np.ma.getmask(mean_masked)] = np.nan

            return mean

        # Median
        elif strategy == "median":
            if tuple(int(v) for v in np.__version__.split('.')[:2]) < (1, 5):
                # In old versions of numpy, calling a median on an array
                # containing nans returns nan. This is different is
                # recent versions of numpy, which we want to mimic
                masked_X.mask = np.logical_or(masked_X.mask,
                                              np.isnan(X))
            median_masked = np.ma.median(masked_X, axis=axis)
            # Avoid the warning "Warning: converting a masked element to nan."
            median = np.ma.getdata(median_masked)
            median[np.ma.getmaskarray(median_masked)] = np.nan

            return median

        # Most frequent
        elif strategy == "most_frequent":
            # scipy.stats.mstats.mode cannot be used because it will no work
            # properly if the first element is masked and if its frequency
            # is equal to the frequency of the most frequent valid element
            # See https://github.com/scipy/scipy/issues/2636

            # To be able access the elements by columns
            if axis == 0:
                X = X.transpose()
                mask = mask.transpose()

            most_frequent = np.empty(X.shape[0])

            for i, (row, row_mask) in enumerate(zip(X[:], mask[:])):
                row_mask = np.logical_not(row_mask).astype(np.bool)
                row = row[row_mask]
                most_frequent[i] = _most_frequent(row, np.nan, 0)

            return most_frequent
        
        ## ADDS start here
        # Random: in fit: send values that can be selected
        elif strategy == 'random':
            
            not_mask = mask == False
            statistics = []         
            for i in range(X.shape[1]):
                statistics.append(X[:, i][not_mask[:, i]])
            
            return statistics #for each column a vector of possible imputations
                
        elif strategy in ['regression', 'stochastic']:
                  
            X = X[~np.isnan(X).any(axis=1)] #how to do this by making use of mask/not_mask?            
            statistics = []
            
            lr = LinearRegression()
            for i in range(X.shape[1]):
                dependent_var = X[:, i]
                independent_var = np.delete(X, i, axis = 1)
                lr.fit(independent_var, dependent_var)
                stats_per_column = [lr.intercept_, lr.coef_]
                
                if strategy == 'stochastic':
                    res = dependent_var - lr.predict(independent_var)
                    sd = np.std(res)
                    stats_per_column.append(sd)
                    
                statistics.append(stats_per_column)
                
            # this is the code from strategy == 'mean'
            mean_masked = np.ma.mean(masked_X, axis=axis)
            # Avoid the warning "Warning: converting a masked element to nan."
            mean = np.ma.getdata(mean_masked)
            mean[np.ma.getmask(mean_masked)] = np.nan
            statistics.append(mean)                       
            
            return statistics #for each column a the fit of a regression line

    def transform(self, X):
        """Impute all missing values in X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            The input data to complete.
        """
        if self.strategy in ['mean', 'median', 'most_frequent']:
            
            if self.axis == 0:
                check_is_fitted(self, 'statistics_')
                X = check_array(X, accept_sparse='csc', dtype=FLOAT_DTYPES, force_all_finite=False, copy=self.copy)
                statistics = self.statistics_
                
                if X.shape[1] != statistics.shape[0]:
                    raise ValueError("X has %d features per sample, expected %d"
                                     % (X.shape[1], self.statistics_.shape[0]))

            # Since two different arrays can be provided in fit(X) and
            # transform(X), the imputation data need to be recomputed
            # when the imputation is done per sample
            else:
                X = check_array(X, accept_sparse='csr', dtype=FLOAT_DTYPES,
                                force_all_finite=False, copy=self.copy)

                if sparse.issparse(X):
                    statistics = self._sparse_fit(X,
                                                  self.strategy,
                                                  self.missing_values,
                                                  self.axis)

                else:
                    statistics = self._dense_fit(X,
                                                 self.strategy,
                                                 self.missing_values,
                                                 self.axis)

            # Delete the invalid rows/columns        
            invalid_mask = np.isnan(statistics)
            valid_mask = np.logical_not(invalid_mask)
            valid_statistics = statistics[valid_mask]
            valid_statistics_indexes = np.where(valid_mask)[0]
            missing = np.arange(X.shape[not self.axis])[invalid_mask]

            if self.axis == 0 and invalid_mask.any():
                if self.verbose:
                    warnings.warn("Deleting features without "
                                  "observed values: %s" % missing)
                X = X[:, valid_statistics_indexes]
            elif self.axis == 1 and invalid_mask.any():
                raise ValueError("Some rows only contain "
                                 "missing values: %s" % missing)

            # Do actual imputation
            if sparse.issparse(X) and self.missing_values != 0:
                mask = _get_mask(X.data, self.missing_values)
                indexes = np.repeat(np.arange(len(X.indptr) - 1, dtype=np.int),
                                    np.diff(X.indptr))[mask]

                X.data[mask] = valid_statistics[indexes].astype(X.dtype,
                                                            copy=False)
            else:
                if sparse.issparse(X):
                    X = X.toarray()
                mask = _get_mask(X, self.missing_values)
                n_missing = np.sum(mask, axis=self.axis)
                values = np.repeat(valid_statistics, n_missing)

                if self.axis == 0:
                    coordinates = np.where(mask.transpose())[::-1]
                else:
                    coordinates = mask

                X[coordinates] = values
                
            return X            
            
        elif self.strategy == 'random':
                    
            if self.axis == 0:
                check_is_fitted(self, 'statistics_')
                X = check_array(X, accept_sparse='csc', dtype=FLOAT_DTYPES, force_all_finite=False, copy=self.copy)
                statistics = self.statistics_       
                
            #Do actual imputation                  
            if sparse.issparse(X):
                X = X.toarray()
            mask = _get_mask(X, self.missing_values)
            n_missing = np.sum(mask, axis=self.axis)
                     
            values = np.array([])
            for i in range(len(statistics)):
                imputations = np.random.choice(statistics[i],size=n_missing[i],replace=True)
                values = np.append(values, imputations)
            
            if self.axis == 0:
                coordinates = np.where(mask.transpose())[::-1]
            else:
                coordinates = mask

            X[coordinates] = values
            
            return X
            
        elif self.strategy in ['regression', 'stochastic']:
                
            if self.axis == 0:
                check_is_fitted(self, 'statistics_')
                X = check_array(X, accept_sparse='csc', dtype=FLOAT_DTYPES, force_all_finite=False, copy=self.copy)
                statistics = self.statistics_  
                mean_values = statistics[-1]
                statistics = statistics[:-1]
                
            #Do actual imputation                  
            if sparse.issparse(X):
                X = X.toarray()
            mask = _get_mask(X, self.missing_values)
            
            values = np.array([])
            for i in range(len(statistics)):
                               
                dependent_var = X[:, i]                          
                independent_var = np.delete(X[mask[:, i]], i, axis=1)
                independent_mask = np.delete(mask[mask[:, i]], i, axis=1)
                
                # this is the code from mean imputation
                # we need complete data to calculate the predictions
                temp_n_missing = np.sum(independent_mask, axis=self.axis)
                temp_mean_values = np.delete(mean_values, i)
                temp_values = np.repeat(temp_mean_values, temp_n_missing)
                coordinates = np.where(independent_mask.transpose())[::-1]
                independent_var[coordinates] = temp_values
                
                imputations = np.dot(independent_var, statistics[i][1]) + statistics[i][0]
                if self.strategy == 'stochastic':
                    noise = np.random.normal(0, statistics[i][2], size = len(imputations))
                    imputations = imputations + noise
                    
                values = np.append(values, imputations)
            
            if self.axis == 0:
                coordinates = np.where(mask.transpose())[::-1]
            else:
                coordinates = mask

            X[coordinates] = values
            
            return X  

In [6]:
class custom_pipeline(Pipeline):
    def transform(self, X):
        assert(isinstance(X,pd.DataFrame)), 'X needs to be pd.DataFrame'
        return pd.DataFrame(super().transform(X),columns = X.columns, index=X.index)

def create_custom_pipeline(imputationstrategy, model):
    pipe = custom_pipeline([
        ('imputer',imputationstrategy),
        ('model',model)])
    return pipe

def create_standard_pipeline(method, model):
    pipe = Pipeline([
        ('imputer',Imputer(strategy=method)),
        ('model',model)])
    return pipe

In [7]:
def apply_mean_median(train,test,model,method='mean'):
    train_use = train.copy()
    test_use = test.copy()
    pipe = create_standard_pipeline(method=method,model=model)
    mod = pipe.fit(train_use.drop(output_variable,1),train_use[output_variable])
    pred = mod.predict(test_use.drop(output_variable,1))
    return {'mse': mse(test_use[output_variable],pred), 'ev': ev(test_use[output_variable],pred),
           'mae': mae(test_use[output_variable],pred)}

In [8]:
def apply_custom_imputation(train,test,model,method='random'):
    train_use = train.copy()
    test_use = test.copy()
    pipe = create_custom_pipeline(imputationstrategy=custom_imputer(strategy=method),model=model)
    mod = pipe.fit(train_use.drop(output_variable,1),train_use[output_variable])
    pred = mod.predict(test_use.drop(output_variable,1))
    return {'mse': mse(test_use[output_variable],pred), 'ev': ev(test_use[output_variable],pred),
            'mae': mae(test_use[output_variable],pred)}

## Simulation function

In [27]:
def perform_simulation(df,n_simulations,missing_proportions,output_variable):
    results_mse = []
    results_ev = []
    results_mae = []
    for simulation in range(n_simulations):
        print('busy with simulation ', simulation)
        incomplete_data = {}
        for mechanism in ['MCAR','MAR','MNAR']:
            incomplete_data[mechanism] = {}
            for proportion in missing_proportions:
                incomplete_data[mechanism][proportion] = \
                delete_data(df_,mechanism,output_variable,missing_row_proportion=proportion).\
                reset_index().drop('index',1)
        
        train_ind, test_ind = apply_train_test(df)
        for missing_type in incomplete_data.keys():
            for missing_perc in incomplete_data[missing_type].keys():
                #scaler = StandardScaler()
                train = incomplete_data[missing_type][missing_perc].loc[train_ind].copy()
                #scaler.fit(train)
                #train = scaler.transform(train) 
                #test = scaler.transform(incomplete_data[missing_type][missing_perc].loc[test_ind].copy())
                test = incomplete_data[missing_type][missing_perc].loc[train_ind].copy()
                missing_cells_proportion = round(train.isnull().sum().mean()/len(train),3)
                for name,model in [['lin',LinearRegression()]]: #['svr', SVR()]]:
                    drop_ = apply_drop(train,test,model)
                    mean_ = apply_mean_median(train,test,model,method='mean')
                    median_ = apply_mean_median(train,test,model,method='median')
                    random_ = apply_custom_imputation(train,test,model,method='random')   
                    regression_ = apply_custom_imputation(train,test,model,method='regression')   
                    stochastic_ = apply_custom_imputation(train,test,model,method='stochastic')  
                                
                    results_mse.append([simulation,missing_type,missing_perc,missing_cells_proportion,name,
                                        drop_['mse'],mean_['mse'],median_['mse'],
                                        random_['mse'],regression_['mse'], stochastic_['mse']])
                    results_mae.append([simulation,missing_type,missing_perc, missing_cells_proportion,name,
                                       drop_['mae'],mean_['mae'],median_['mae'],
                                       random_['mae'],regression_['mae'],stochastic_['mae']])
                    results_ev.append([simulation,missing_type,missing_perc, missing_cells_proportion,name,
                                       drop_['ev'],mean_['ev'],median_['ev'],
                                       random_['ev'],regression_['ev'],stochastic_['ev']])
                #del scaler
    return results_mse, results_ev, results_mae

def save_results(results_mse,results_ev,results_mae,nsimulations,missing_proportions):
    export_mse = pd.DataFrame(results_mse,columns = ['simulation','missing_type','missing_rows_proportion',
                                                     'missing_cells_proportion','model',
                                                     'drop','mean','median','random','regression','stochastic'])
    export_ev = pd.DataFrame(results_ev,columns = ['simulation','missing_type','missing_rows_proportion',
                                                   'missing_cells_proportion','model',
                                                   'drop','mean','median','random','regression','stochastic'])
    export_mae = pd.DataFrame(results_mae,columns = ['simulation','missing_type','missing_rows_proportion',
                                                   'missing_cells_proportion','model',
                                                   'drop','mean','median','random','regression','stochastic'])
    export = pd.concat([export_mse, export_mae, export_ev])
    rows_one_metric = n_simulations * len(missing_proportions) * 3
    export['evaluation_metric'] = np.append(np.repeat('mse', rows_one_metric),
                                            np.append(np.repeat('mae', rows_one_metric),
                                                      np.repeat('ev', rows_one_metric)))

    group_dict = {}
    for col in ['drop','mean','median','random', 'regression','stochastic']:
        group_dict[col] = 'mean'
        export[col+'_upper'] = export[col].copy()
        export[col+'_lower'] = export[col].copy()
        group_dict[col+'_lower'] = lambda x: np.percentile(x,25)
        group_dict[col+'_upper'] = lambda x: np.percentile(x,75)
    group_dict['missing_cells_proportion'] = 'mean'

    export = export.groupby(['missing_type','missing_rows_proportion','model','evaluation_metric']).agg(group_dict)
    
    return export    

## Simulation with real dataset: Forest Fires

In [18]:
missing_proportions = np.linspace(0.05,0.55,20)
n_simulations = 50

In [19]:
df_ = pd.read_csv('Data/forest_fires.txt',sep='\t')
output_variable = 'area'
df_.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [28]:
#results_mse, results_ev, results_mae = perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse,results_ev,results_mae,n_simulations,missing_proportions)

In [31]:
export

drop  \
missing_type missing_rows_proportion model evaluation_metric                
MAR          0.050000                lin   ev                    0.030789   
                                           mae                  20.175572   
                                           mse                4196.294795   
             0.076316                lin   ev                    0.028744   
                                           mae                  19.919139   
                                           mse                4182.132230   
             0.102632                lin   ev                    0.030188   
                                           mae                  19.697820   
                                           mse                3993.570931   
             0.128947                lin   ev                    0.030501   
                                           mae                  19.967116   
                                           mse                4190.601793   
             0.155263                lin   ev                    0.030625   
                                           mae                  19.284370   
                                           mse                3918.256673   
             0.181579                lin   ev                    0.032847   
                                           mae                  19.198388   
                                           mse                3763.857446   
             0.207895                lin   ev                    0.032595   
                                           mae                  18.529988   
                                           mse                3601.537226   
             0.234211                lin   ev                    0.029523   
                                           mae                  19.718362   
                                           mse                4268.219465   
             0.260526                lin   ev                    0.035035   
                                           mae                  18.821094   
                                           mse                3662.134095   
             0.286842                lin   ev                    0.032328   
                                           mae                  18.787326   
                                           mse                3702.743124   
             0.313158                lin   ev                    0.033448   
                                           mae                  18.506263   
                                           mse                3713.527578   
             0.339474                lin   ev                    0.034819   
                                           mae                  17.908492   
                                           mse                3537.352012   
             0.365789                lin   ev                    0.032247   
                                           mae                  17.793380   
                                           mse                3647.780750   
             0.392105                lin   ev                    0.036972   
                                           mae                  18.603255   
                                           mse                3796.102034   
             0.418421                lin   ev                    0.039014   
                                           mae                  18.445046   
                                           mse                3626.240999   
             0.444737                lin   ev                    0.036956   
                                           mae                  18.226617   
                                           mse                3792.966652   
             0.471053                lin   ev                    0.041331   
                                           mae                  17.660595   
                                           mse                342

In [32]:
export.to_csv('Results/results_forest_fires.txt',sep='\t')

## Simulation with real dataset: Slump Test

In [26]:
df_ = pd.read_csv('Data/slump_test.txt',sep='\t')
output_variable = 'SLUMP(cm)'
df_.head()

,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm)
0,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0
1,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0
2,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0
3,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0
4,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0


In [27]:
results_mse, results_ev = perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_ev,n_simulations,missing_proportions)
export.to_csv('Results/results_slump_test.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 

busy with simulation  320
busy with simulation  321
busy with simulation  322
busy with simulation  323
busy with simulation  324
busy with simulation  325
busy with simulation  326
busy with simulation  327
busy with simulation  328
busy with simulation  329
busy with simulation  330
busy with simulation  331
busy with simulation  332
busy with simulation  333
busy with simulation  334
busy with simulation  335
busy with simulation  336
busy with simulation  337
busy with simulation  338
busy with simulation  339
busy with simulation  340
busy with simulation  341
busy with simulation  342
busy with simulation  343
busy with simulation  344
busy with simulation  345
busy with simulation  346
busy with simulation  347
busy with simulation  348
busy with simulation  349
busy with simulation  350
busy with simulation  351
busy with simulation  352
busy with simulation  353
busy with simulation  354
busy with simulation  355
busy with simulation  356
busy with simulation  357
busy with si

## Simulation with custom dataset

In [28]:
n_simulations = 50

In [29]:
#custom_dataset_poor_small
df_ = pd.read_csv('Data/custom_dataset_poor_small.txt',sep='\t')
output_variable = 'y'
results_mse, results_ev = perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_ev,n_simulations,missing_proportions)
export.to_csv('Results/results_custom_dataset_poor_small.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 

In [30]:
#custom_dataset_poor_medium
df_ = pd.read_csv('Data/custom_dataset_poor_medium.txt',sep='\t')
output_variable = 'y'
results_mse, results_ev = perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_ev,n_simulations,missing_proportions)
export.to_csv('Results/results_custom_dataset_poor_medium.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 

In [31]:
#custom_dataset_poor_large
df_ = pd.read_csv('Data/custom_dataset_poor_large.txt',sep='\t')
output_variable = 'y'
results_mse, results_ev = perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_ev,n_simulations,missing_proportions)
export.to_csv('Results/results_custom_dataset_poor_large.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 

In [32]:
#custom_dataset_rich_small
df_ = pd.read_csv('Data/custom_dataset_rich_small.txt',sep='\t')
output_variable = 'y'
results_mse, results_ev = perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_ev,n_simulations,missing_proportions)
export.to_csv('Results/results_custom_dataset_rich_small.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 

In [33]:
#custom_dataset_rich_medium
df_ = pd.read_csv('Data/custom_dataset_rich_medium.txt',sep='\t')
output_variable = 'y'
results_mse, results_ev = perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_ev,n_simulations,missing_proportions)
export.to_csv('Results/results_custom_dataset_rich_medium.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 

In [34]:
#custom_dataset_rich_large
df_ = pd.read_csv('Data/custom_dataset_rich_large.txt',sep='\t')
output_variable = 'y'
results_mse, results_ev = perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_ev,n_simulations,missing_proportions)
export.to_csv('Results/results_custom_dataset_rich_large.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 